# Lecture 3: RecSys Algorithms and Pipeline

## Overview

The goal of this assignment is to get you familiar with some recommender systems algorithms and apply them to a sample of the steam dataset. 
The dataset is provided as a parquet file in the `data/` directory of this repository. 
It is simply a subset of the `train_interactions.csv` file on blackboard where random users are dropped.

You will first implement cosine similarity and the ItemKNN algorithm. Then, you will build a pipeline for lightGBM using the predictions from ItemKNN as candidates.

Throughout this Jupyter Notebook, any code that requires your input or completion will be clearly marked with a Task and a [TODO] tag. Your job is to fill in these sections with the appropriate code, following the instructions and hints provided to guide you through each step.

By the end of this task, you will have:

   - Implemented cosine similarity 
   - Implemented the ItemKNN algorithm
   - Evaluated the ItemKNN algorithm
   - Implemented a recommender pipeline
   - Generated and understood candidates
   - Implemented some features
   - Gotten to know LightGBM
   - Evaluated your Hybrid model

## Setup

We first import some libraries and define helper functions.

In [6]:
# import os
# os.chdir("/home/david/Desktop/recommenderSystems")


import scipy as sp
import numpy as np
import pandas as pd
from lightgbm import LGBMRanker
from recpack.matrix import InteractionMatrix
from recpack.scenarios import StrongGeneralization
from recpack.util import get_top_K_ranks
from src.metrics import calculate_ndcg, calculate_calibrated_recall
import warnings

warnings.simplefilter("ignore", sp.sparse.SparseEfficiencyWarning)


# helper function to turn a sparse matrix into a dataframe
def matrix2df(X) -> pd.DataFrame:
    coo = sp.sparse.coo_array(X)
    return pd.DataFrame({
        "user_id": coo.row,
        "item_id": coo.col,
        "value": coo.data
    })


# helper function to convert a score matrix into a dataframe of recommendations
def scores2recommendations(
    scores: sp.sparse.csr_matrix, 
    X_test_in: sp.sparse.csr_matrix, 
    recommendation_count: int,
    prevent_history_recos = True
) -> pd.DataFrame:
    # ensure you don't recommend fold-in items
    if prevent_history_recos:
        scores[(X_test_in > 0)] = 0
    # rank items
    ranks = get_top_K_ranks(scores, recommendation_count)
    # convert to a dataframe
    df_recos = matrix2df(ranks).rename(columns={"value": "rank"}).sort_values(["user_id", "rank"])
    return df_recos

ModuleNotFoundError: No module named 'src'

### Dataset loading and splitting

We start by loading in the sampled Steam dataset and splitting it using strong generalization. 
You should be familiar with strong generalization and dataset splitting from the previous assignment.

For simplicity, we use the strong generalization scenario implementation from Recpack to get a train-test split, with 80% of the data being used for training and 20% for testing. 
As a reminding, strong generalization means that we don't have any overlap between the users of the test and training set.
The test sets consists of a fold-in part (containing the history of the test users) and a hold-out part (containing the remaining "ground truth" interactions of those users).

**The goal is to use the training data to create a model, give it the fold-in test data as input, and have it predict the hold-out interactions.**

[An example](https://recpack.froomle.ai/generated/recpack.scenarios.StrongGeneralization.html#recpack.scenarios.StrongGeneralization) can be found in the documentation of recpack.

In [1]:
#print(os.path.abspath(os.getcwd()))
# load data from parquet file
df = pd.read_parquet("lectures/data/interactions.parquet", engine="fastparquet")

print(df.head())

# create a Recpack interaction matrix
X = InteractionMatrix(df, item_ix="item_id", user_ix="user_id")

# split matrix using strong generalization
# we use a fixed seed for reproducibility
scenario = StrongGeneralization(frac_users_train=0.8, frac_interactions_in=0.8, validation=False, seed=42)
scenario.split(X)

# get interaction matrices
X_train = scenario.full_training_data.values
X_test_in = scenario.test_data_in.values
X_test_out = scenario.test_data_out.values

# dataframe version of hold-out set to compute metrics later on
df_test_out = matrix2df(X_test_out)

NameError: name 'pd' is not defined

## Task 1: Implementing ItemKNN

### Objective

Implement ItemKNN using cosine as similarity measure.

### Overview

1. We will first implement cosine similarity so we can calculate all pairwise item similarities from an interaction matrix
2. Next, we will implement ItemKNN to calculate scores for each (user, item) pair
3. Finally, we will generate recommendations based on the scores and calculate some evaluation metrics

### Background

The ItemKNN algorithm consists of 3 main steps and makes recommendations based on the similarities between items.

The first step is to define and implement a similarity measure between items. In this task we will use cosine similarity, but there exist many more. For those interested, additional similarities can be found in the slides and on the web. The second step is to find the k most similar neighbors for each item (i.e. prune the similarity matrix). The final step is to calculate a score for each (user, item) pair.

### Task 1.1: Cosine similarity

Complete the function in the cell below to calculate all pairwise cosine similarities.
Items should not be considered similar to themselves, so remember to set the diagonal to zero.

The cosine similarity between two vectors $\bf{a}$ and $\bf{b}$ is given by: 

$$\cos\left(\bf{a},\bf{b}\right)=\dfrac{\bf{a}\cdot\bf{b}}{\| \bf{a}\|_2 \cdot \| \bf{b}\|_2 }=\dfrac{\sum_{i=0}^{n} a_i \cdot b_i}{\sqrt{\sum_{i=0}^{n} a_i^2} \cdot \sqrt{\sum_{i=0}^{n} b_i^2}}$$

**Hint**: think about how you can compute this for all pairwise similarities at once.

In [23]:
def my_cosine_similarity(X: sp.sparse.csr_matrix) -> sp.sparse.csr_matrix:
    # TODO: implement cosine similarity function
    # hint: should return a sparse (item x item)-matrix
    # hint: work with the entire matrix to benefit from highly optimized vectorized operations in numpy and scipy
    # important: don't forget to set the diagonal to 0 (no self-similarity)

    dot_product = X.T.dot(X)

    norms = np.sqrt(X.power(2).sum(axis=0)).A1
    norms[norms == 0] = 1e-10  # avoid division by zero

    inv_norms = 1.0 / norms
    cosine_sim = dot_product.multiply(inv_norms).T.multiply(inv_norms).T

    cosine_sim.setdiag(0)

    # removing small values from the matrix
    cosine_sim.eliminate_zeros()
    cosine_sim.data[np.abs(cosine_sim.data) < 1e-12] = 0
    cosine_sim.eliminate_zeros()

    return cosine_sim.tocsr()


# check whether similarities match expected output
S = my_cosine_similarity(X_train)
assert sp.sparse.isspmatrix(S)
assert S.shape == (8523, 8523)
print(S.nnz)
assert S.nnz == 11817736
assert np.isclose(S[0, 0], 0)
assert np.isclose(S[0, 42], 0.1145873121273334)
assert np.isclose(S[42, 314], 0.06482037234831424)

11817736


### Task 1.2: Complete ItemKNN implementation

Complete the `item_knn_scores` function in the cell below to calculate recommendation scores based on the ItemKNN algorithm.
You should use the `get_top_K_values` function from [repack](https://gitlab.com/recpack-maintainers/recpack/-/blob/master/recpack/util.py?ref_type=heads#L80) to prune the similarities and keep only the k highest values for each row.

The code below the function uses the scores to create a dataframe of recommendations.
This dataframe is compared against the hold-out dataframe using NDCG and Recall as accuracy metrics.

The expected outputs are:

+ NDCG@10: 0.26746
+ Recall@10: 0.27379

Note that the results may differ due to non-determinism in `get_top_K_values` and `get_top_K_ranks`

In [24]:
from recpack.util import get_top_K_values

def item_knn_scores(
    X_train: sp.sparse.csr_matrix, 
    X_test_in: sp.sparse.csr_matrix, 
    neighbor_count: int
) -> sp.sparse.csr_matrix:
    # TODO: add code to compute scores for all (user, item) pairs
    # hint: use your own cosine similarity function
    # hint: use `get_top_K_values` to prune the similarity matrix
    # hint: think about how you can calculate the scores for all pairs at once
    S = my_cosine_similarity(X_train)

    top_k = get_top_K_values(S, neighbor_count)

    scores = X_test_in.dot(top_k)

    return scores

scores = item_knn_scores(X_train, X_test_in, 50)
df_recos = scores2recommendations(scores, X_test_in, 10)

ndcg = calculate_ndcg(df_recos, 10, df_test_out)
recall = calculate_calibrated_recall(df_recos, 10, df_test_out)

print(f"  NDCG@10: {ndcg:.5f}")
print(f"Recall@10: {recall:.5f}")

  NDCG@10: 0.26727
Recall@10: 0.27367


## Task 2: LightGBM Pipeline

### Objective

Extend and use a recommendation pipeline that generates candidates, adds features and ranks the candidates.

### Overview

1. First, we generate candidates based on popularity and your itemKNN implementation
2. Next, we add the features from your previous assignment
3. Finally, we use LightGBMRanker to rank the candidates and evaluate the results

### Background

You can read more about LightGBM and recommendation pipelines in [this](https://medium.com/@mohtasim.hossain2000/mastering-lightgbm-an-in-depth-guide-to-efficient-gradient-boosting-8bfeff15ee17) and [this](https://medium.com/data-science/building-a-recommender-system-using-machine-learning-2eefba9a692e) medium article.


### Task 2.1: Candidate generation

The first step is candidate generation.

For the training candidates we start with all true interactions (positive examples), we then add the most popular items as extra candidates. Some of these popular item candidate will coincide with true interaction that we already added, this is not a problem and we just combine duplicates (automatic with `pd.merge`). Some other popular item candidates will not be true interaction and these become our negative examples.

The LGBMRanker will use these positive en negative examples to learn a model that can rank candidates for which we don't know whether they are positive of negative: i.e. our test candidates.

Our test candidates should be generated as similarly to our training candidates as possible. The major difference is that we do not include any interactions from the fold-in. After all, we know the fold-in and hold-out are disjoint.

**Task**: Add candidates based on your ItemKNN implementation and include a feature indicating that the candidate originates from ItemKNN.

#### Tips

+ Look at how the popularity candidates are generated for guidance
    + The popularity candidates are the same for each user but your ItemKNN candidates will be personalized
    + The popularity rank is added as a feature for the candidates, you can do something similar for your candidates
    + The `pd.merge` operations will look nearly identical.
+ Use `scores2recommendations` to get a dataframe based on your ItemKNN scores
    + For the training candidates, you don't want to remove repeat recommendation (`prevent_history_recos=False`)
. Otherwise the model will learn that ItemKNN candidates are never relevant, which we don't want.
    + For the test candidate you do want to remove repeat recommendation (`prevent_history_recos=True`)
+ Don't forget to adjust the `pd.fillna` at the end for your new feature.

In [25]:
def get_candidates_popularity(X_train, X_test_in, k=10):
    item_counts = X_train.sum(axis=0).A1
    top_items = np.argpartition(item_counts, -k)[-k:][::-1]

    train_users = np.where(X_train.sum(axis=1).A1)[0]
    test_users = np.where(X_test_in.sum(axis=1).A1)[0]

    df_candidates_train = pd.DataFrame(data={
        "user_id": train_users,
        "item_id": list(np.tile(top_items, (len(train_users), 1)))
    }).explode("item_id")

    df_candidates_test = pd.DataFrame(data={
        "user_id": test_users,
        "item_id": list(np.tile(top_items, (len(test_users), 1)))
    }).explode("item_id")
    
    df_candidates_train["popularity_rank"] = df_candidates_train.groupby("user_id").cumcount()
    df_candidates_test["popularity_rank"] = df_candidates_test.groupby("user_id").cumcount()

    return df_candidates_train, df_candidates_test

# generate candidates
df_train = matrix2df(X_train).rename(columns={"value": "played"})
df_test = pd.DataFrame(columns=["user_id", "item_id"], dtype=np.int64) # empty dataframe

# add popularity candidates
df_candidates_pop_train, df_candidates_pop_test = get_candidates_popularity(X_train, X_test_in, 10)
df_train = pd.merge(df_train, df_candidates_pop_train, how="outer", on=["user_id", "item_id"])
df_test = pd.merge(df_test, df_candidates_pop_test, how="outer", on=["user_id", "item_id"])

# add item knn candidates
df_candidates_knn_train = scores2recommendations(
    S, X_train, recommendation_count=10, prevent_history_recos=False
)

df_candidates_knn_test = scores2recommendations(
    S, X_test_in, recommendation_count=10, prevent_history_recos=False
)

df_train = pd.merge(df_train, df_candidates_knn_train, how="outer", on=["user_id", "item_id"])
df_test = pd.merge(df_test, df_candidates_knn_test, how="outer", on=["user_id", "item_id"])

df_candidates_knn_train["itemknn_rank"] = df_candidates_knn_train.groupby("user_id").cumcount()
df_candidates_knn_test["itemknn_rank"] = df_candidates_knn_test.groupby("user_id").cumcount()

# fill missing values
df_train.fillna({"played": 0, "popularity_rank": 999, "itemknn_rank": 999}, inplace=True)
df_test.fillna({"popularity_rank": 999, "itemknn_rank": 999}, inplace=True)


### Task 2.2: Create features

We now have our candidates, but we don't have any features except those related to the origin of the candidates. For our model to improve, we need to add content-based features related to the items and or users.

**Task**: 

+ Create 2 features to the candidates based on the previous assignments.
+ Add them to both `df_test` and `df_train`.
+ Explain your features too, explain why you think they might help to better rank the candidates.

#### Example

An example feature might be the price of the game. You will need to make sure to first preprocess and clean the price values because LightGBM can only handle numerical values (no strings or datetime objects). Once you have the price for each game, you can merge this data into your candidate dataframes (e.g. `pd.merge(df_train, df_item_price, on="item_id")`). Make sure add the features to both `df_test` and `df_train`.

In [ ]:
df_games = pd.read_csv("data/games.csv")
df_games["price"].head()
df_games.head()

FileNotFoundError: [Errno 2] No such file or directory: 'cleaned_datasets_students/games.csv'

In [ ]:
df_extended_games = pd.read_csv("data/extended_games.csv")
df_extended_games.T.head(40)


,0,1,2,3,4,5,6,7,8,9,...,6743,6744,6745,6746,6747,6748,6749,6750,6751,6752
item_id,0,1,2,3,13,14,16,17,19,20,...,8510,8513,8514,8515,8516,8517,8519,8520,8521,8522
item_name,Counter-Strike,Rag Doll Kung Fu,Silo 2,Call of Duty: World at War,Runespell: Overture,Dead Mountaineer's Hotel,Vertex Dispenser,PT Boats: Knights of the Sea,PT Boats: South Gambit,Orcs Must Die!,...,Yar's Revenge,Renegade Ops,Blade Kitten,Garshasp: The Monster Slayer,Garshasp: Temple of the Dragon,Haunted House,NightSky,The UnderGarden,Spiral Knights,Puzzle Pirates
release_date,"Nov 1, 2000","Oct 12, 2005","Dec 19, 2012","Nov 18, 2008","Jul 20, 2011","Oct 28, 2011","Jun 10, 2011","Oct 28, 2011","Oct 28, 2011","Oct 11, 2011",...,"Apr 28, 2011","Oct 26, 2011","May 22, 2014","May 9, 2011","Sep 24, 2012","Oct 12, 2023","Mar 1, 2011","Nov 10, 2010","Jun 14, 2011","Aug 31, 2011"
required_age,0,0,0,17,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
price,9.99,0.99,59.99,19.99,9.99,7.99,9.99,6.99,6.99,9.99,...,9.99,15.0,2.99,4.99,4.99,19.99,9.99,1.39,0.0,0.0
dlc_count,0,0,0,0,0,0,0,0,0,2,...,0,2,3,0,0,0,0,0,1,0
detailed_description,Play the world's number 1 online action game. ...,A piece of Steam history - THE FIRST EVER NON ...,Note: This is a legacy version of Silo which i...,"Call of Duty is back, redefining war like you'...",Runespell: Overture is a role-playing game com...,The remote hotel stands atop an ominous snow-p...,Vertex Dispenser is an abstract real-time stra...,PT Boats is dedicated to a small group of litt...,"“The seas can be dangerous, but when you’re in...","Slice them, burn them, skewer them, and launch...",...,"In Yar’s Revenge, take flight as you explore e...",Destruction just got awesome. In Renegade Ops ...,The Hollow Wish Collection Only available here...,Years after the confinement of Azhi Dahaka by ...,"The mighty mythological hero, Garshasp, travel...",Chills and stealthy thrills abound In Haunted ...,"Nominated as a IGF Seamus McNally Finalist, Ni...",The UnderGarden is a casual Zen game that chal...,Band together and fight to the Core! Spiral Kn...,Brace yourself for swashbuckling puzzle action...
about_the_game,Play the world's number 1 online action game. ...,A piece of Steam history - THE FIRST EVER NON ...,Note: This is a legacy version of Silo which i...,"Call of Duty is back, redefining war like you'...",Runespell: Overture is a role-playing game com...,The remote hotel stands atop an ominous snow-p...,Vertex Dispenser is an abstract real-time stra...,PT Boats is dedicated to a small group of litt...,"“The seas can be dangerous, but when you’re in...","Slice them, burn them, skewer them, and launch...",...,"In Yar’s Revenge, take flight as you explore e...",Destruction just got awesome. In Renegade Ops ...,The Hollow Wish Collection Only available here...,Years after the confinement of Azhi Dahaka by ...,"The mighty mythological hero, Garshasp, travel...",Chills and stealthy thrills abound In Haunted ...,"Nominated as a IGF Seamus McNally Finalist, Ni...",The UnderGarden is a casual Zen game that chal...,Band together and fight to the Core! Spiral Kn...,Brace yourself for swashbuckling puzzle action...
short_description,Play the world's number 1 online action game. ...,A piece of Steam history - THE FIRST EVER NON ...,"Silo is a focused, lightning-fast standalone 3...","Call of Duty is back, redefining war like you'...","This role-playing game, set in an alternate me...",The remote hotel stands atop an ominous snow-p...,#app_102400_short_desc,PT Boats is dedicated to a small group of litt...,"“The seas can be dangerous, but when you’re in...","Slice them, burn them, skewer them, and launch...",...,Take flight as you explore exotic alien worlds...,Blast your way through enemy lines to defeat I...,"In Blade Kitten, join Kit Ballard as she explo...","As the legendary Persian hero, fight your way ...","The mighty mythological hero, Garshasp, travel...","Creep, sneak, and dash your way through hordes...","Nominated as a IGF Seamus McNally Finali

In [ ]:
# add two features to df_train and df_test
df_extended_games['positive_ratings_ratio'] = df_extended_games['positive'] / (df_extended_games['positive'] + df_extended_games['negative'])
df_extended_games['positive_ratings_ratio'].head(50)

df_train = pd.merge(df_train, df_extended_games[['item_id', 'positive_ratings_ratio']], left_on='item_id', right_on='item_id', how='left')
df_test = pd.merge(df_test, df_extended_games[['item_id', 'positive_ratings_ratio']], left_on='item_id', right_on='item_id', how='left')

print(df_train.columns)

Index(['user_id', 'item_id', 'played', 'popularity_rank', 'rank',
       'positive_ratings_ratio_x', 'price', 'positive_ratings_ratio_y'],
      dtype='object')


In [ ]:

# Data preprocessing: replacing the Free, Try this for Free, etc. values with 0
non_numeric_prices = df_games[~df_games['price'].astype(str).str.replace('.', '', 1).str.isdigit()]
non_numeric_prices['price'].unique()
# we can convert it to a 0
df_games['price'] = df_games['price'].replace(non_numeric_prices['price'].unique(), '0')
# Let's also convert the Nan value to 0
df_games['price'] = pd.to_numeric(df_games['price'], errors='coerce').fillna(0)

df_train = pd.merge(df_train, df_games[['item_id', 'price']], left_on='item_id', right_on='item_id', how='left')
df_test = pd.merge(df_test, df_games[['item_id', 'price']], left_on='item_id', right_on='item_id', how='left')

Your features should be new columns in the candidate dataframes. The columns of both dataframes should match except for `df_test` which doesn't have the `played` column.

In [40]:
print(df_train.columns)
print(df_test.columns)

Index(['user_id', 'item_id', 'played', 'popularity_rank', 'rank',
       'positive_ratings_ratio', 'price'],
      dtype='object')
Index(['user_id', 'item_id', 'popularity_rank', 'rank',
       'positive_ratings_ratio', 'price'],
      dtype='object')


### Task 2.3: Ranking

The template below is almost completely filled in, you just need to specify which columns to use as features.

You should not include `user_id`, `item_id` or `played` as features.

The output shows the importance of features and the NDCG and Recall of the final recommendations.

Did your features improve the recommendations? Feel free to play around with the pipeline and see if you can further improve the results. Try adding more candidates or changing the parameters of the lightGBM model.

In [41]:
feature_cols = ["price", "popularity_rank", "rank", "positive_ratings_ratio"]

# define ranker with sensible default parameters
ranker = LGBMRanker(
    force_row_wise=True,
    objective="lambdarank",
    metric="ndcg",
    boosting_type="dart",
    n_estimators=100,
    importance_type='gain',
)

# specify which records belong together
df_train = df_train.sort_values("user_id")
train_groups = df_train.groupby("user_id")["item_id"].count().to_list()

# fit ranker
ranker.fit(
    df_train[feature_cols], df_train["played"],
    group=train_groups
)

# generate final recommendations
df_test["score"] = ranker.predict(df_test[feature_cols])
df_recos = df_test.sort_values(["user_id", "score"], ascending=[True, False])

# evaluate recommendations
ndcg = calculate_ndcg(df_recos, 10, df_test_out)
recall = calculate_calibrated_recall(df_recos, 10, df_test_out)

# print results
print("------------------")
print(f"  NDCG@10: {ndcg:.5f}")
print(f"Recall@10: {recall:.5f}")

# print feature importance
for i in ranker.feature_importances_.argsort()[::-1]:
    imp = ranker.feature_importances_[i] / ranker.feature_importances_.sum()
    print(f"{feature_cols[i]:>30} {imp:.5f}")

[LightGBM] [Info] Total Bins 343
[LightGBM] [Info] Number of data points in the train set: 418353, number of used features: 4
------------------
  NDCG@10: 0.15730
Recall@10: 0.18627
               popularity_rank 0.72412
                          rank 0.24459
        positive_ratings_ratio 0.02639
                         price 0.00491


## Congratulations! You’ve completed the assignment.